<H1>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;YAPAY ZEKA DERSİ PROJESİ</H1>

# Filmlerin İzleyici Yorumlarının İncelenmesi ve Analiz Edilmesi


## NLP (Natural Language Processing)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Yapay Zeka İşlemleri
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import nltk
from nltk.corpus import stopwords

# Veri temizleme işlemleri
from bs4 import BeautifulSoup
import re # Regular Expression

# Train test işlemleri
from sklearn.model_selection import train_test_split




In [2]:
# Yorumların bulunduğu Verilerimizi yüklüyoruz
df = pd.read_csv('NLPlabeledData.tsv',  delimiter="\t", quoting=3)



In [3]:
# Veri tablomuz
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
len(df)

25000

In [5]:
len(df["review"]) # verimizde toplam 25k yorumumuz var

25000

In [6]:
# İngilizce gramer kelimelerini temizlemek için kullanıyoruz. (The, am, is, are vb.)
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\duhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Duygu analizi için verimi temizlemem gerekiyor.

### Html taglari, noktalama işaretleri, büyük harf vs. kaldırılmalı

In [7]:
sample_review= df.review[0]
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [8]:
# HTML tagleri temizleme işlemi.
sample_review = BeautifulSoup(sample_review).get_text()
sample_review

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [9]:
# Noktalama işaretleri ve sayıları temizleme.
sample_review = re.sub("[^a-zA-Z]",' ',sample_review)
sample_review

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [10]:
# küçük harfe dönüştürüyoruz, 
# Makine öğrenmesi büyük harfleri farklı kelime olarak algılayabilir.
sample_review = sample_review.lower()
sample_review

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [11]:
# stopwords (yani the, is, are gibi gramer kelimelerine ihtiyacımız yok)

# önce split ile kelimeleri bölüyoruz (Aksi takdirde stopwords çalışmıyor )ve listeye dönüştürüyoruz.
sample_review = sample_review.split()


  

In [12]:
sample_review

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [13]:
len(sample_review)

437

In [14]:
# sample_review without stopwords
swords = set(stopwords.words("english"))                      # conversion into set for fast searching
sample_review = [w for w in sample_review if w not in swords]               
sample_review


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [15]:
len(sample_review)

219

In [16]:
# Temizleme işlemini açıkladıktan sonra şimdi tüm dataframe'imiz içinde bulunan reviewleri döngü içinde topluca temizliyoruz

In [19]:
def process(review):
    # HTML tags için
    review = BeautifulSoup(review).get_text()
    # Harf ve noktalama işaretleri
    review = re.sub("[^a-zA-Z]",' ',review)
    # Tamamı küçük harf.
    review = review.lower()
    # stopwords
    review = review.split()
    swords = set(stopwords.words("english"))                      # conversion into set for fast searching
    review = [w for w in review if w not in swords]               
    # splitted paragraph'ları space ile birleştiriyoruz return
    return(" ".join(review))

In [20]:
# training datamızı yukarıdaki fonksiyonu kullanarak temizliyoruz.

train_x_tum = []
for r in range(len(df["review"])):        
    if (r+1)%25000 == 0:        
        print("Temizlenen veri :", r+1)
    train_x_tum.append(process(df["review"][r]))

Temizlenen veri : 25000


### Train, test split...

In [21]:
x = train_x_tum
y = np.array(df["sentiment"]) # Duygu

# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1)


### Bag of Words oluşturuyoruz

Verilerimizi temizledik ancak yapay zekanın çalışması için bu metin tabanlı verileri sayılara ve bag of words denilen matrise çevirmek gerekiyor. İşte bu amaçla sklearn içinde bulunan CountVectorizer aracını kullanıyoruz:

In [22]:
train_x

['disappointing irritating screenwriter true understanding human nature instead strings together clich disjointed fashion character explored depth puppets plunked plot openly confesses needed dramatic element mother affair daughter lover anne reid gives excellent performance spite given peculiar situations lines standing passively allow angry daughter slug eye script portrays darren daniel craig dr jekyll mr hyde without hints except snorts cocaine big scene mature writer could done much topic',
 'town royston vasey weird wonderful place characters would wrong disturbing fantastically brilliant writing means works works well people know others touch characters hopefully one knows people extremes personalities tubbs edward stranger hating owners local shop pen obsessed pauline treats dole scum much contempt strange inhabitants tv works consists series christmas special references many horror films wicker man recent addition range works film league gentlemens apocalypse say much highly r

In [23]:
# sklearn içinde bulunan countvectorizer fonksiyonunu kullanarak max 5000 kelimelik bag of words oluşturuyoruz...
vectorizer = CountVectorizer( max_features = 5000 )

# train verilerimizi feature vektöre matrisine çeviriyoruz
train_x = vectorizer.fit_transform(train_x)



In [24]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1776043 stored elements in Compressed Sparse Row format>

In [25]:
# Bunu array'e dönüştürüyoruz çünkü fit işlemi için array istiyor..
train_x = train_x.toarray()
train_y = y_train

In [26]:
train_x.shape, train_y.shape

((22500, 5000), (22500,))

In [27]:
train_y

array([0, 1, 1, ..., 1, 0, 1], dtype=int64)

### Random Forest Modeli oluşturuyoruz ve fit ediyoruz. Böylece train işlemimiz bitiyor


In [28]:
model = RandomForestClassifier(n_estimators = 100) # Duyguları anlamak için ağaç sayım 100
model.fit(train_x, train_y)


RandomForestClassifier()

### Şimdi sıra test datamızda..

In [29]:
test_x

['want see mystery watch though elements straight elmore leonard territory comes closer episode dynasty since filmmaker focuses character development e long boring talks stupid un involving characters people make fascinating movies without real action see exotica one avoid especially like actors involved one',
 'young john travolta trilogy blockbusters dances disco rock n roll country heads houston find work love gilley hot spot time mechanical bull outdone rode bull club nashville recently saw nearly forgotten film television remembered good good year wore black cowboy hat year like travolta debra winger prime looks stunning red top plenty charisma bud sissy seem ideal couple even trailer trash split feels good getting back together urban cowboy amazing soundtrack get hear lyin eyes eagles lookin love johnny lee',
 'warning spoilers typical disney film policticly correct foster home even divison races insults viewers intellect simplistic lines boy slezy father almost directly taken ne

In [30]:
# Test verilerimizi feature vektöre matrisine çeviriyoruz
# Bag of Words
test_xx = vectorizer.transform(test_x)


In [31]:
test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 198915 stored elements in Compressed Sparse Row format>

In [32]:
test_xx = test_xx.toarray() # Tahmin işlemi için array olması gerekiyor.

In [33]:
test_xx.shape

(2500, 5000)

#### Tahmin işlemimizi yapıyoruz.

In [34]:
test_predict = model.predict(test_xx) # test_predict bana test_y verilerimi veriyor.
dogruluk = roc_auc_score(y_test, test_predict)

In [35]:
print("Doğruluk oranı : % ", dogruluk * 100)

Doğruluk oranı : %  84.98345015576324
